In [25]:
import numpy as np
import gym
import gym_pen
import os

def read_data(file_path, file_name):
    all_xy_arrays = []
    xy_array = []
    xy_coord = [0,0]
    with open(os.path.join(file_path, file_name), 'r') as file:
        pen_mode = 0
        is_start_box = False
        for line in file:
            if line == '.START_BOX\n':
                if is_start_box:
                    if len(xy_array) > 0:
                        all_xy_arrays.append(xy_array)
                    xy_array = []
                is_start_box = True
                pen_mode = 1

            if is_start_box:
                if line == '.PEN_UP\n' or line == '.PEN_DOWN\n':
                    pen_mode = 0
                else:
                    try:
                        xy_coord = line.replace('\n','').replace('\t',' ').split(' ')
                        xy_coord = [int(item) for item in xy_coord if item != '']
                        xy_coord.append(pen_mode)
                        xy_array.append(xy_coord)
                        pen_mode = 1
                    except:
                        pass
        # 마지막 array도 저장
        if len(xy_array) > 0:
            all_xy_arrays.append(xy_array)
    return all_xy_arrays


def scale_data(xy_array, shape):
    min_x = np.min(xy_array[:, 0])
    max_x = np.max(xy_array[:, 0])
    min_y = np.min(xy_array[:, 1])
    max_y = np.max(xy_array[:, 1])

    xy_array_scaled = np.copy(xy_array)
    xy_array_scaled[:, 0] = (shape[0] - 1) * (xy_array[:, 0] - min_x) / (max_x - min_x)
    xy_array_scaled[:, 1] = (shape[1] - 1) * (xy_array[:, 1] - min_y) / (max_y - min_y)

    return xy_array_scaled


In [26]:

file_path = 'unipen/CDROM/train_r01_v07/include/aga/data/'
file_lst = os.listdir(file_path)
for file_name in file_lst:
    all_xy_arrays = read_data(file_path, file_name) 
    for idx, xy_array in enumerate(all_xy_arrays):
        try:
            xy_array = np.array(xy_array, dtype= np.intc)
            min_x = np.min(xy_array[:, 0])
            max_x = np.max(xy_array[:, 0])
            min_y = np.min(xy_array[:, 1])
            max_y = np.max(xy_array[:, 1])

            shape = (max_x - min_x, max_y - min_y)

            scaled_xy_array = scale_data(xy_array, shape)

            # PenEnv 환경을 생성합니다. 원하는 shape를 인자로 넘겨서 환경을 초기화합니다.
            env = gym.make('pen-v0', shape=shape)

            # 환경 초기화
            observation, _ = env.reset()

            # 스케일된 좌표를 이용하여 환경 실행
            for i in range(len(scaled_xy_array)):
                env.step(scaled_xy_array[i])

            directory = 'images/'
            if not os.path.exists(directory):
                os.makedirs(directory)

            save_path = f'images/{file_name[:-4]}_{idx}.jpg'
            env.save(save_path)
        except:
            pass


/var/folders/65/fdpvqph137z6ypl4n_tx4qyw0000gn/T/ipykernel_2757/2373917917.py:47: RuntimeWarning: invalid value encountered in divide
  xy_array_scaled[:, 0] = (shape[0] - 1) * (xy_array[:, 0] - min_x) / (max_x - min_x)
/var/folders/65/fdpvqph137z6ypl4n_tx4qyw0000gn/T/ipykernel_2757/2373917917.py:47: RuntimeWarning: invalid value encountered in cast
  xy_array_scaled[:, 0] = (shape[0] - 1) * (xy_array[:, 0] - min_x) / (max_x - min_x)
/var/folders/65/fdpvqph137z6ypl4n_tx4qyw0000gn/T/ipykernel_2757/2373917917.py:48: RuntimeWarning: invalid value encountered in divide
  xy_array_scaled[:, 1] = (shape[1] - 1) * (xy_array[:, 1] - min_y) / (max_y - min_y)
/var/folders/65/fdpvqph137z6ypl4n_tx4qyw0000gn/T/ipykernel_2757/2373917917.py:48: RuntimeWarning: invalid value encountered in cast
  xy_array_scaled[:, 1] = (shape[1] - 1) * (xy_array[:, 1] - min_y) / (max_y - min_y)
/opt/homebrew/anaconda3/envs/ml/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:64: UserWarning: WARN: A Box a